Steps:
1. Make a beautiful soup object from homepage to extract the links of postable locations
2. Get zipcodes for each location
3. Create a dataframe with location, link, zipcode

In [20]:
def makesoup(url):
    import requests, bs4, re
    res = requests.get(url)
    res.raise_for_status()
    soup = bs4.BeautifulSoup(res.text, "lxml")
    return(soup)

In [21]:
#Create bs object from clist homepage in order to extract the links of all postable locations
url = 'https://newyork.craigslist.org/'
soup = makesoup(url)
link_info = soup.find('li', {'class':'s'} )

In [22]:
# Clean up the link_info using regex to get rid of everything but the urls and locations
import re
pat = re.compile('"\/\/(\w+\.\w+\.org\/">\w+\s?\w+)')
m = re.findall(pat,str(link_info))
with open('clistlinks.txt', 'w') as wf:
    for item in m:
        wf.write(item)
        wf.write('\n')

30

1

36

1

36

...

1

36

1

26

1

In [23]:
#Form lists for the urls and locations
urls = []
locs = []
with open('clistlinks.txt', 'r') as rf:
    for line in rf.readlines():
        l=line.split('"')
        urls.append(l[0])
        locs.append(l[1].replace('"',"").replace('>','').replace('\n',''))
#can use mult chars to split on

In [25]:
#Add prefix to urls 
links=[]
for item in urls:
    new="https://"+item
    links.append(new)

In [26]:
# Create dataframe with the locations and the links
import pandas as pd
df = pd.DataFrame(links,index=locs,columns=['link'])

In [27]:
df

,link
albany,https://albany.craigslist.org/
allentown,https://allentown.craigslist.org/
baltimore,https://baltimore.craigslist.org/
binghamton,https://binghamton.craigslist.org/
catskills,https://catskills.craigslist.org/
central nj,https://cnj.craigslist.org/
delaware,https://delaware.craigslist.org/
eastern ct,https://newlondon.craigslist.org/
eastern shore,https://easternshore.craigslist.org/
harrisburg,https://harrisburg.craigslist.org/


In [31]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException, NoSuchElementException, UnexpectedAlertPresentException, WebDriverException
from selenium.webdriver.chrome.options import Options

In [30]:
# Get zip codes for each location and create dictionary connecting zip codes to location
driver = webdriver.Chrome()
d={}
for elem in locs:
    driver.get("http://zipcode.org/zip_code_search")
    e = driver.find_element_by_name("txtTownCity")
    e.clear()
    e.send_keys(elem)
    e.send_keys(Keys.RETURN)
    try:
        content = driver.find_element_by_class_name('Search_Results_Link_Text')
        zipcode = content.get_property("text")
        d[elem]=zipcode
    except:
        d[elem]='unknown'
    print(zipcode)
    zipcode=''
driver.close()
#update to include extra landing page, use case/switch

05820
08501
12124
13901


07833


17101
05047

17740
01523
04050
05472


13820
13673

01864

18501



17701
01601
03909


In [32]:
d

{'albany': '05820',
 'allentown': '08501',
 'baltimore': '12124',
 'binghamton': '13901',
 'catskills': 'unknown',
 'central nj': 'unknown',
 'delaware': '07833',
 'eastern ct': 'unknown',
 'eastern shore': 'unknown',
 'harrisburg': '17101',
 'hartford': '05047',
 'hudson valley': 'unknown',
 'jersey shore': '17740',
 'lancaster': '01523',
 'long island': '04050',
 'new haven': '05472',
 'north jersey': 'unknown',
 'northwest ct': 'unknown',
 'oneonta': '13820',
 'philadelphia': '13673',
 'poconos': 'unknown',
 'reading': '01864',
 'rhode island': 'unknown',
 'scranton': '18501',
 'south coast': 'unknown',
 'south jersey': 'unknown',
 'western mass': 'unknown',
 'williamsport': '17701',
 'worcester': '01601',
 'york': '03909'}

In [53]:
good=[item for item in d.items() if item[1] != 'unknown']
good

[('baltimore', '12124'),
 ('allentown', '08501'),
 ('long island', '04050'),
 ('worcester', '01601'),
 ('delaware', '07833'),
 ('williamsport', '17701'),
 ('scranton', '18501'),
 ('oneonta', '13820'),
 ('york', '03909'),
 ('new haven', '05472'),
 ('hartford', '05047'),
 ('harrisburg', '17101'),
 ('reading', '01864'),
 ('binghamton', '13901'),
 ('philadelphia', '13673'),
 ('jersey shore', '17740'),
 ('lancaster', '01523'),
 ('albany', '05820')]

In [54]:
# Group together those that failed
fails=[key for key,val in d.items() if val=='unknown']

In [80]:
fails
len(fails)

['south coast',
 'north jersey',
 'western mass',
 'central nj',
 'eastern ct',
 'south jersey',
 'northwest ct',
 'hudson valley',
 'eastern shore',
 'rhode island']

10

In [63]:
for elem in fails[1:]:
    driver = webdriver.Chrome()
    driver.get("http://www.google.com")
    e = driver.find_element_by_name("q")
    e.clear()
    e.send_keys(elem +' zip code')
    e.send_keys(Keys.RETURN)
    driver.find_element_by_class('title')    
    break

AttributeError: 'WebDriver' object has no attribute 'find_element_by_class'

In [74]:
import re
with open('zipcodes.txt','r')as rf:
        f_contents = rf.read()
string = f_contents
for fail in fails:
    try:
        pattern = fail+ ' \d+'
        matchobj = re.search(pattern, string, re.I|re.M).group(0)
        m = matchobj.split()
        d[m[0]]=m[1]
    except:
        print(fail)

south coast
north jersey
western mass
central nj
eastern ct
south jersey
northwest ct
hudson valley
eastern shore
rhode island


In [81]:
#See if there are any other zips still missing 
fails=[key for key,val in d.items() if val=='unknown']
fails
len(fails)

['south coast',
 'north jersey',
 'western mass',
 'central nj',
 'eastern ct',
 'south jersey',
 'northwest ct',
 'hudson valley',
 'eastern shore',
 'rhode island']

10

In [ ]:
#Add missing zips to dictionary by hand
d['binghamton']='13901'
d['catskills']='12414'
d['boston']='02110'
d['north jersey']='07024'
d['eastern ct']='06226'
d['south coast']='02790'
d['elmira']='14901'

In [ ]:
#Ensure that the length of dictionary matches the number of rows in the dataframe
len(d)==df.shape[0]

In [ ]:
#Add zip codes to dataframe
df['zip_codes']=d.values()
df.index.name = 'location'
df.columns = ['url','zip_code']

In [10]:
import pickle
df= pickle.load(open('clist_df','rb'))

In [16]:
#save dataframe to pickle to easily restore later
df.to_pickle('clist_df')

Next steps:
3. Use Selenium to post an ad to a particular location using a particular username and password
4. Post several ads in multiple locations by looping through various username/password combinations